<a href="https://colab.research.google.com/github/renatojmf/Aspects-Extraction-in-Portuguese/blob/main/CAt_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-Processamento

In [ ]:
! pip install stanza
! pip install unidecode

In [ ]:
from google.colab import files
uploaded = files.upload()

In [2]:
import stanza as st
import pandas as pd
st.download('pt') # download pt model
nlp = st.Pipeline('pt') # initialize pt neural pipeline

2022-03-30 16:14:34 INFO: Downloading default packages for language: pt (Portuguese)...


2022-03-30 16:14:43 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-03-30 16:14:43 INFO: Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |
| depparse  | bosque  |

2022-03-30 16:14:43 INFO: Use device: cpu
2022-03-30 16:14:43 INFO: Loading: tokenize
2022-03-30 16:14:43 INFO: Loading: mwt
2022-03-30 16:14:43 INFO: Loading: pos
2022-03-30 16:14:44 INFO: Loading: lemma
2022-03-30 16:14:44 INFO: Loading: depparse
2022-03-30 16:14:46 INFO: Done loading processors!


In [ ]:
import json
tvs = json.load(open("/content/tv.json"))

In [ ]:
texts = []
aspectsClass = []
for x in tvs['reviews']:
  texts.append(x['review'])
  y = x['implicit aspects']
  if(len(y) > 0):
    aspectsClass.append(y[0])
  else:
    aspectsClass.append(None)

d = {'text': texts, 'aspectsClass':aspectsClass}

In [ ]:
print(len(texts))
print(len(aspectsClass))
for x in aspectsClass[0:20]:
  print(x)

1091
1091
entrega
produto
None
funcionalidades
None
None
None
None
None
None
None
produto
produto
desempenho
produto
peso
None
None
produto
None


In [ ]:
df_tvs = pd.DataFrame(data=d)

In [ ]:
df_tvs.head()

,text,aspectsClass
0,Excelente smart tv. E foi entregue vem antes d...,entrega
1,"Imagem ótima recursos de web bons , recomendo ...",produto
2,"produto ótimo, entrega antes do prazo previsto...",None
3,"Muito bom o produto, esta tv está entre as das...",funcionalidades
4,a entrega foi super rápido... muito antes do p...,None


In [ ]:
values = [None, 'None']
df = df_tvs[df_tvs.aspectsClass.isin(values) == False]

In [ ]:
df.head()

,text,aspectsClass
0,Excelente smart tv. E foi entregue vem antes d...,entrega
1,"Imagem ótima recursos de web bons , recomendo ...",produto
3,"Muito bom o produto, esta tv está entre as das...",funcionalidades
11,Todos os aparelhos samsung são de uma qualidad...,produto
12,"Atendeu todas as expectativas, recomendo Samsu...",produto


In [ ]:
print(len(df_tvs),(len(df)))

1091 595


In [58]:
df = pd.read_csv('/content/restaurantes.csv',sep=';')

#passar apenas os textos/frases para a função
def getCoNLLU(df):
  dicts = []
  for texto in df:
    doc = nlp(texto) # doc is class Document
    dicts.append(doc.to_dict())

  return dicts

In [59]:
from unidecode import unidecode
import re

words = []

for word in df['text']:
  old_string = unidecode(word)
  new_string = re.sub(r"[^a-zA-Z0-9]"," ",old_string)
  words.append(new_string.lower())

In [60]:
df['text'] = words

In [61]:
df.head()

,text,aspects,aspectsClass
0,hamburgueria artesanal com tempero diferenciad...,tempero,Food
1,excelente peixe para quem esta localizado no ...,peixe,Food
2,serve lanches e cerveja gelada tem mesinhas f...,cerveja,Drinks
3,muito bom o atendimento,atendimento,Service
4,cardapio variado,Cardápio,Food


In [ ]:
dicts = getCoNLLU(df['text'])

saida = ''
for sentenca in dicts[0:2]:
  for tokens in sentenca:
    for tks in tokens:
      for tag in tks:
        #print("tag:" ,tag)
        saida += str(tks[tag])  + "\t"
      saida +="\n"
    saida +="\n"
  saida +="\n"


print(saida)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


1	excelente	excelente	ADJ	Gender=Fem|Number=Sing	2	amod	0	9	
2	smart	smart	X	7	nsubj	10	15	
3	tv	tv	NOUN	Gender=Fem|Number=Sing	2	compound	16	18	
4	e	e	CCONJ	6	cc	20	21	
5	foi	ser	AUX	Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin	6	aux:pass	22	25	
6	entregue	entregar	VERB	Gender=Masc|Number=Sing|VerbForm=Part|Voice=Pass	2	conj	26	34	
7	vem	vir	VERB	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	0	root	35	38	
8	antes	antes	ADV	7	advmod	39	44	
(9, 10)	do	45	47	
9	de	de	ADP	11	case	
10	o	o	DET	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	11	det	
11	prazo	prazo	NOUN	Gender=Masc|Number=Sing	8	obl	48	53	
12	informado	informar	VERB	Gender=Masc|Number=Sing|VerbForm=Part	11	acl	54	63	


1	imagem	imagem	NOUN	Gender=Fem|Number=Sing	7	nsubj	0	6	
2	otima	otimo	ADJ	Gender=Fem|Number=Sing	3	amod	7	12	
3	recursos	recurso	NOUN	Gender=Masc|Number=Plur	1	nmod	13	21	
4	de	de	ADP	5	case	22	24	
5	web	web	NOUN	Gender=Masc|Number=Sing	3	nmod	25	28	
6	bons	bom	ADJ	Gender=Masc|Number=Plur	5	

In [ ]:
def getNouns():
  for sentenca in dicts:
    for tokens in sentenca:
      for tks in tokens:
        for tag in tks:
          if tks[tag] == 'NOUN':
            yield (tks['text'].lower())
          

In [ ]:
from collections import Counter, defaultdict
import json

c = Counter()
c.update(getNouns())
nouns = getNouns()
print(c)

json.dump(dict(c), open('/content/nouns.json', 'w'))

Counter({'tv': 255, 'produto': 245, 'qualidade': 140, 'imagem': 139, 'prazo': 134, 'som': 105, 'entrega': 98, 'expectativas': 65, 'internet': 44, 'samsung': 38, 'custo': 38, 'netflix': 35, 'preco': 35, 'compra': 30, 'youtube': 28, 'americanas': 28, 'audio': 26, 'smart': 24, 'aparelho': 24, 'controle': 24, 'parabens': 23, 'aplicativos': 23, 'funcoes': 19, 'vezes': 19, 'marca': 19, 'expectativa': 18, 'submarino': 18, 'problema': 17, 'pena': 16, 'acesso': 14, 'sinal': 14, 'televisao': 14, 'volume': 14, 'funcao': 14, 'instalacao': 13, 'problemas': 13, 'loja': 13, 'quarto': 13, 'tempo': 13, 'so': 12, 'dias': 12, 'manuseio': 12, 'data': 11, 'nota': 11, 'modelo': 11, 'uso': 11, 'recursos': 10, 'aquisicao': 10, 'coisa': 10, 'lojas': 10, 'necessidades': 10, 'produtos': 10, 'sistema': 10, 'certeza': 10, 'filmes': 10, 'parede': 10, 'resto': 9, 'tamanho': 9, 'atendimento': 9, 'estado': 9, 'momento': 9, 'defeito': 9, 'dia': 9, 'rapidez': 8, 'casa': 8, 'configuracao': 8, 'conexao': 8, 'ponto': 8, 'c

In [ ]:
def getCorpus2(df):
  corpus = []
  for item in df:
    corpus.append(item.split())
  
  return corpus

In [ ]:
from gensim.models import Word2Vec
corpus = getCorpus2(df['text'])
#print(corpus[1::])

f = Word2Vec(corpus[1::],
                 sg=0,
                 negative=5,
                 window=10,
                 size=200,
                 min_count=2,
                 iter=5,
                 workers=10)

f.wv.save_word2vec_format("/content/my_word_vectors.vec")


In [ ]:
d = json.load(open('/content/nouns.json'))
nouns2 = Counter()
#print(f.wv.vocab.items[0])
for k, v in d.items():
  for tag,_ in f.wv.vocab.items():
    if k.lower() == tag:
      #print("valor v: ",v)
      nouns2[k.lower()] += v

# nouns2, _ = zip(*sorted(nouns2.items(),
#                 key=lambda x: x[1],
#                 reverse=True))

# json.dump(nouns, open("/content/aspect_words.json", "w"))

In [ ]:
nouns3, _ = zip(*sorted(nouns2.items(),
                key=lambda x: x[1],
                reverse=True))

json.dump(nouns3, open("/content/aspect_words.json", "w"))

# Métricas e Funções do algoritmo

In [28]:
! pip install reach

  Created wheel for reach: filename=reach-3.4.7-py3-none-any.whl size=8533 sha256=bf9c1fe40fd351139adfa2b44c51b39de451e92d54bc71cf0c5e3e646546b38f
  Stored in directory: /root/.cache/pip/wheels/16/f7/16/14cec58071c89f16832ff04929412108d9c3dba0b989c8798c
Successfully built reach


In [29]:
import json
from reach import Reach
from collections import defaultdict
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import rbf_kernel
from collections import Counter
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score,accuracy_score

In [30]:
def normalize(x):
    """Normalize a vector while controlling for zero vectors."""
    x = np.copy(x)
    if np.ndim(x) == 1:
        norm = np.linalg.norm(x)
        if norm == 0:
            return x
        return x / np.linalg.norm(x)
    norm = np.linalg.norm(x, axis=-1)
    mask = norm > 0
    x[mask] /= norm[mask][:, None]
    return x


In [31]:
"""Simple method."""
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import rbf_kernel
from collections import Counter


def get_aspects(fragments, embeddings, n_adj_seed, n_nouns, min_count):
    """Get aspects based on fragments."""
    adj, _, noun = zip(*fragments)
    adj_cand, _ = zip(*Counter(adj).most_common(n_adj_seed))

    cands = candidate(embeddings,
                      adj,
                      noun,
                      adj_cand,
                      n_nouns,
                      min_count)

    return cands


def candidate(embeddings,
              adj,
              noun,
              seed_words,
              n_nouns,
              min_count):
    """
    Generates candidate aspects based on adjective co-occurrences

    Parameters
    ----------
    embeddings : Reach
        A Reach instance containing the word embeddings.
    constructions : list of tuples
        A list of adjective noun tuples.
    seed_words : list of str
        A list of strings. All these words should be in vocab for the
        given embeddings model.
    frequency_threshold : int
        Any noun occurring fewer times than this threshold is discarded
    n_nouns : int
        The amount of items to return

    Returns
    -------
    candidates : dict
        A dictionary mapping strings to their scores.

    """
    a = list(set(adj))
    sims = embeddings.similarity(a, seed_words).max(1)
    adj_scores = dict(zip(a, sims))

    noun_scores = defaultdict(lambda: [0, 0])
    for adj, noun in zip(adj, noun):
        noun_scores[noun][0] += adj_scores[adj]
        noun_scores[noun][1] += 1

    noun_scores = {k: v[0] for k, v in noun_scores.items()
                   if v[1] > min_count}

    return sorted(noun_scores.items(), key=lambda x: x[1])[-n_nouns:]


def rbf_attention(vec, memory, gamma, **kwargs):
    """
    Single-head attention using RBF kernel.

    Parameters
    ----------
    vec : np.array
        an (N, D)-shaped array, representing the tokens of an instance.
    memory : np.array
        an (M, D)-shaped array, representing the memory items
    gamma : float
        the gamma of the RBF kernel.

    Returns
    -------
    attention : np.array
        A (1, N)-shaped array, representing a single-headed attention mechanism

    """
    z = rbf_kernel(vec, memory, gamma)
    s = z.sum()
    if s == 0:
        # If s happens to be 0, back off to uniform
        return np.ones((1, len(vec))) / len(vec)
    return (z.sum(1) / s)[None, :]


def softmax(x, axis=1):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x, axis, keepdims=True))
    s = e_x.sum(axis=axis, keepdims=True)
    return e_x / s


def attention(vec, memory, **kwargs):
    """
    Standard multi-head attention mechanism.

    Parameters
    ----------
    vec : np.array
        an (N, D)-shaped array, representing the tokens of an instance.
    memory : np.array
        an (M, D)-shaped array, representing the memory items

    Returns
    -------
    attention : np.array
        A (M, N)-shaped array, representing the attention over all memories.

    """
    z = memory.dot(vec.T)
    return softmax(z)


def mean(vec, aspect_vecs, **kwargs):
    """Just a mean weighting."""
    return (np.ones(len(vec)) / len(vec))[None, :]

In [32]:
def get_scores(instances,
               aspects,
               r,
               labels,
               remove_oov=False,
               attention_func=attention,
               **kwargs):
    """Scoring function."""
    print(labels)
    assert all([x in r.items for x in labels])
    
    label_vecs = normalize(r.vectorize(labels))
    aspect_vecs = [x.mean(0)
                   for x in r.transform(aspects,
                                        remove_oov=False)]
    aspect_vecs = np.stack(aspect_vecs)
    if len(instances) == 1:
        instances = [instances]

    t = r.transform(instances, remove_oov=remove_oov)

    out = []
    for vec in t:
        att = attention_func(vec, aspect_vecs, **kwargs)
        #print("att",att)
        # Att = (n_heads, n_words)
        z = att.dot(vec)
        # z = (n_heads, n_dim)
        x = normalize(z).dot(label_vecs.T)
        # x = (n_heads, n_labels)
        out.append(x.sum(0))
    return np.stack(out)

# Avaliação com o dataset restaurantes.csv

## Processamento e escolhas de labels para os dados

In [62]:
df['labels'] =-1
df.head()

,text,aspects,aspectsClass,labels
0,hamburgueria artesanal com tempero diferenciad...,tempero,Food,-1
1,excelente peixe para quem esta localizado no ...,peixe,Food,-1
2,serve lanches e cerveja gelada tem mesinhas f...,cerveja,Drinks,-1
3,muito bom o atendimento,atendimento,Service,-1
4,cardapio variado,Cardápio,Food,-1


In [63]:
df['aspectsClass'].unique()

array(['Food', 'Drinks', 'Service', 'Ambience', ' Price', 'Location',
       'Price', 'Miscellaneous', 'General', nan, 'service'], dtype=object)

In [64]:
df.loc[df["aspectsClass"] == "Service", "labels"] = 0
df.loc[df["aspectsClass"] == "Food", "labels"] = 1
df.loc[df["aspectsClass"] == "Price", "labels"] = 2
df.loc[df["aspectsClass"] == "Ambience", "labels"] = 3
df.loc[df["aspectsClass"] == "Location", "labels"] = 4

In [65]:
df_tst = df[df['labels'] != -1]
df_tst.head()

,text,aspects,aspectsClass,labels
0,hamburgueria artesanal com tempero diferenciad...,tempero,Food,1
1,excelente peixe para quem esta localizado no ...,peixe,Food,1
3,muito bom o atendimento,atendimento,Service,0
4,cardapio variado,Cardápio,Food,1
5,ambiente agradavel,Ambiente,Ambience,3


In [66]:
y_true = df_tst['labels']

## Avaliações com word2vec cbow_s50

In [ ]:
GAMMA = .03
N_ASPECT_WORDS = 200

scores = defaultdict(dict)
r = Reach.load("/content/cbow_s50.txt",unk_word="<UNK>")

aspects = [[x] for x in json.load(open("/content/aspect_words.json"))]
aspects = aspects[:N_ASPECT_WORDS]
instances = ["a comida estava deliciosa e muito bem temperada".lower().split(), 
              "o atendimento estava incrível, recomendo demais".lower().split(), 
              "um ambiente de otima qualidade".lower().split(),
              "o lugar é muito bem localizado".lower().split(),
              "achei o preco um pouco caro mas vale a pena".lower().split()]
#instances = df['text'][5:10]
print(instances)
label_set = ['atendimento', 'comida', 'preco', 'ambiente', 'lugar']
#[i.lower().split() for i in instances]
s = get_scores(instances,
               aspects,
               r,
               label_set,
               gamma=GAMMA,
               remove_oov=False,
               attention_func=rbf_attention)

pred = s.argmax(1)
#print(s)
print(pred)

[['a', 'comida', 'estava', 'deliciosa', 'e', 'muito', 'bem', 'temperada'], ['o', 'atendimento', 'estava', 'incrível,', 'recomendo', 'demais'], ['um', 'ambiente', 'de', 'otima', 'qualidade'], ['o', 'lugar', 'é', 'muito', 'bem', 'localizado'], ['achei', 'o', 'preco', 'um', 'pouco', 'caro', 'mas', 'vale', 'a', 'pena']]
[1 0 3 4 1]


In [71]:
GAMMA = .03
N_ASPECT_WORDS = 200

scores = defaultdict(dict)
r = Reach.load("/content/cbow_s50.txt",unk_word="<UNK>")

aspects = [[x] for x in json.load(open("/content/aspect_words.json"))]
aspects = aspects[:N_ASPECT_WORDS]
instances = df_tst['text']
# instances = ["a comida estava deliciosa e muito bem temperada".lower().split(), 
#               "o atendimento estava incrível, recomendo demais".lower().split(), 
#               "um ambiente de otima qualidade".lower().split(),
#               "o lugar é muito bem localizado".lower().split(),
#               "achei o preco um pouco caro mas vale a pena".lower().split()]
#instances = df['text'][5:10]
#print(instances)
label_set = ['atendimento', 'comida', 'preco', 'ambiente', 'lugar']
#[i.lower().split() for i in instances]
s = get_scores([i.lower().split() for i in instances],
               aspects,
               r,
               label_set,
               gamma=GAMMA,
               remove_oov=False,
               attention_func=rbf_attention)

y_pred = s.argmax(1)
#print(s)
print(y_pred)

['atendimento', 'comida', 'preco', 'ambiente', 'lugar']
[1 3 0 1 3 1 1 1 1 4 1 1 1 1 1 1 3 1 1 1 1 0 1 1 1 3 3 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 4 1 3 1 1 1 1 0 1 3 1 1 1 1 0 0 0 0 1 3 3 1 3 1 1 1 3 1 3 1 2 1 0
 1 1 1 1 3 1 1 1 3 0 4 1 1 1 1 0 0 1 1 3 1 1 1 1 3 0 2 1 2 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 3 1 1 1 0 3 1 0 1 1 0 0 4 1 4 0 3 1 1 1 1 4 1 3 0 1 1 1 1 1 1
 1 3 1 1 1 1 1 1 4 1 1 1 1 4 3 3 3 1 1 1 1 1 1 1 3 1 1 1 1 1 1 3 1 1 1 1 1
 3 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1]


In [76]:
accuracy_score(y_true, y_pred)

0.527363184079602

In [73]:
print(classification_report(y_true, y_pred,target_names=label_set))

              precision    recall  f1-score   support

 atendimento       0.74      0.33      0.45        43
      comida       0.53      0.93      0.68        82
       preco       0.67      0.11      0.18        19
    ambiente       0.36      0.25      0.29        40
       lugar       0.50      0.24      0.32        17

    accuracy                           0.53       201
   macro avg       0.56      0.37      0.38       201
weighted avg       0.55      0.53      0.47       201



In [74]:
f1_score = precision_recall_fscore_support(y_true, y_pred)
f1_macro = precision_recall_fscore_support(y_true,y_pred,average="weighted")
print(f1_score)
print()
print(f1_macro)

(array([0.73684211, 0.53146853, 0.66666667, 0.35714286, 0.5       ]), array([0.3255814 , 0.92682927, 0.10526316, 0.25      , 0.23529412]), array([0.4516129 , 0.67555556, 0.18181818, 0.29411765, 0.32      ]), array([43, 82, 19, 40, 17]))

(0.5508309007916236, 0.527363184079602, 0.47499582950827673, None)


## Avaliação com my_word_vectors gerado no pré-processamento do dataset

In [67]:
GAMMA = .03
N_ASPECT_WORDS = 200

scores = defaultdict(dict)
r = Reach.load("/content/my_word_vectors.vec",unk_word="<UNK>")

aspects = [[x] for x in json.load(open("/content/aspect_words.json"))]
aspects = aspects[:N_ASPECT_WORDS]
instances = ["a comida estava deliciosa e muito bem temperada".lower().split(), 
              "o atendimento estava incrível, recomendo demais".lower().split(), 
              "um ambiente de otima qualidade".lower().split(),
              "o lugar é muito bem localizado".lower().split(),
              "achei o preco um pouco caro mas vale a pena".lower().split()]
#instances = df['text'][5:10]
print(instances)
label_set = ['atendimento', 'comida', 'preco', 'ambiente', 'lugar']
#[i.lower().split() for i in instances]
s = get_scores(instances,
               aspects,
               r,
               label_set,
               gamma=GAMMA,
               remove_oov=False,
               attention_func=rbf_attention)

pred = s.argmax(1)
print(s)
print(pred)

[['a', 'comida', 'estava', 'deliciosa', 'e', 'muito', 'bem', 'temperada'], ['o', 'atendimento', 'estava', 'incrível,', 'recomendo', 'demais'], ['um', 'ambiente', 'de', 'otima', 'qualidade'], ['o', 'lugar', 'é', 'muito', 'bem', 'localizado'], ['achei', 'o', 'preco', 'um', 'pouco', 'caro', 'mas', 'vale', 'a', 'pena']]
['atendimento', 'comida', 'preco', 'ambiente', 'lugar']
[[0.9547266  0.95819604 0.9406847  0.9333397  0.94423556]
 [0.9700805  0.94707143 0.9397762  0.93460137 0.94361514]
 [0.943849   0.9407705  0.94308615 0.9519595  0.94289553]
 [0.94415426 0.9342752  0.94304526 0.9306154  0.9592436 ]
 [0.9499382  0.9430925  0.9535146  0.9381563  0.9412436 ]]
[1 0 3 4 2]


In [78]:
GAMMA = .03
N_ASPECT_WORDS = 200

scores = defaultdict(dict)
r = Reach.load("/content/my_word_vectors.vec",unk_word="<UNK>")

aspects = [[x] for x in json.load(open("/content/aspect_words.json"))]
aspects = aspects[:N_ASPECT_WORDS]
instances = df_tst['text']
label_set = ['atendimento', 'comida', 'preco', 'ambiente', 'lugar']
s = get_scores([i.lower().split() for i in instances],
               aspects,
               r,
               label_set,
               gamma=GAMMA,
               remove_oov=False,
               attention_func=rbf_attention)

y_pred = s.argmax(1)
print(y_pred)

['atendimento', 'comida', 'preco', 'ambiente', 'lugar']
[0 4 0 0 3 0 0 0 4 4 0 1 1 1 1 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 2 0 0 2 1 0 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 1 3 0 0 0 3 0 0 0 2 0 0
 0 0 0 1 0 2 0 0 0 0 0 1 0 0 3 0 0 0 0 0 0 0 0 1 0 0 0 1 2 0 0 0 2 0 0 0 0
 0 0 0 0 4 0 0 0 0 0 0 0 1 0 0 0 0 0 4 0 4 0 2 0 0 0 0 4 2 3 0 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 4 4 0 0 0 0 0 0 0 0 0 1 1 2 0 2 0 1 0 0 0 0 0 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [79]:
accuracy_score(y_true, y_pred)

0.3383084577114428

In [80]:
print(classification_report(y_true, y_pred,target_names=label_set))

              precision    recall  f1-score   support

 atendimento       0.27      0.93      0.41        43
      comida       0.62      0.20      0.30        82
       preco       0.55      0.32      0.40        19
    ambiente       0.80      0.10      0.18        40
       lugar       0.22      0.12      0.15        17

    accuracy                           0.34       201
   macro avg       0.49      0.33      0.29       201
weighted avg       0.54      0.34      0.30       201



In [ ]:
f1_score = precision_recall_fscore_support(y_true, y_pred)
f1_macro = precision_recall_fscore_support(y_true,y_pred,average="weighted")
print(f1_score)
print()
print(f1_macro)

(array([0.25899281, 0.64516129, 0.2       , 0.88888889, 0.11764706]), array([0.8372093 , 0.24390244, 0.05263158, 0.2       , 0.11764706]), array([0.3956044 , 0.3539823 , 0.08333333, 0.32653061, 0.11764706]), array([43, 82, 19, 40, 17]))

(0.5243555821367621, 0.3333333333333333, 0.31185122142629174, None)


# Avaliações com o dataset de TVs

## Pré processamento das labels

In [ ]:
df['labels'] =-1
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,text,aspectsClass,labels
0,Excelente smart tv. E foi entregue vem antes d...,entrega,-1
1,"Imagem ótima recursos de web bons , recomendo ...",produto,-1
3,"Muito bom o produto, esta tv está entre as das...",funcionalidades,-1
11,Todos os aparelhos samsung são de uma qualidad...,produto,-1
12,"Atendeu todas as expectativas, recomendo Samsu...",produto,-1


In [ ]:
df['aspectsClass'].unique()

array(['entrega', 'produto', 'funcionalidades', 'desempenho', 'peso',
       'inicialização', 'design', 'usabilidade', 'conexão', 'tamanho',
       'funcionamento', 'imagem', 'manuseio', 'instalação', 'resolução',
       'configuração', 'conexão wifi', 'explorer', 'processamento',
       'consumo', 'entrega, produto', 'entrega:produto',
       'acesso a internet', 'estrutura', 'processador', 'som',
       'velocidade', 'desgin', 'desempenho-', 'tv', 'produto-'],
      dtype=object)

In [ ]:
df.loc[df["aspectsClass"] == "produto", "labels"] = 0
df.loc[df["aspectsClass"] == "entrega", "labels"] = 1
df.loc[df["aspectsClass"] == "imagem", "labels"] = 2
df.loc[df["aspectsClass"] == "desempenho", "labels"] = 3
df.loc[df["aspectsClass"] == "som", "labels"] = 4

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
df_tst = df[df['labels'] != -1]
df_tst.head(10)

,text,aspectsClass,labels
0,Excelente smart tv. E foi entregue vem antes d...,entrega,1
1,"Imagem ótima recursos de web bons , recomendo ...",produto,0
11,Todos os aparelhos samsung são de uma qualidad...,produto,0
12,"Atendeu todas as expectativas, recomendo Samsu...",produto,0
13,"Rapidez, Boa qualidade e preocupação em manter...",desempenho,3
14,"ÓTIMA IMAGEM, FÁCIL ACESSO À INTERNET COM WIFI...",produto,0
18,"Produto Não Deixa nada a desejar, exelente, go...",produto,0
28,Produto excelente!! Recomendo produto alem de ...,entrega,1
29,"Recomendo, tem muitas funções praticas, o wi-f...",produto,0
30,"Maravilhosa.Chegou dentro do prazo estipulado,...",produto,0


In [ ]:
y_true = df_tst['labels']

## Avaliações com word2vec cbow_s50

In [ ]:
GAMMA = .03
N_ASPECT_WORDS = 200

scores = defaultdict(dict)
r = Reach.load("/content/cbow_s50.txt",unk_word="<UNK>")

aspects = [[x] for x in json.load(open("/content/aspect_words.json"))]
aspects = aspects[:N_ASPECT_WORDS]
instances = df_tst['text']
# instances = ["a comida estava deliciosa e muito bem temperada".lower().split(), 
#               "o atendimento estava incrível, recomendo demais".lower().split(), 
#               "um ambiente de otima qualidade".lower().split(),
#               "o lugar é muito bem localizado".lower().split(),
#               "achei o preco um pouco caro mas vale a pena".lower().split()]
#instances = df['text'][5:10]
#print(instances)
label_set = ['produto', 'entrega', 'imagem', 'desempenho', 'som']
#[i.lower().split() for i in instances]
s = get_scores([i.lower().split() for i in instances],
               aspects,
               r,
               label_set,
               gamma=GAMMA,
               remove_oov=False,
               attention_func=rbf_attention)

y_pred = s.argmax(1)
#print(s)
print(y_pred)

[1 2 4 1 2 4 1 0 4 4 0 0 0 4 1 4 0 0 1 4 2 1 2 0 0 1 4 0 2 2 4 4 4 2 4 1 4
 2 1 3 4 4 4 0 4 2 1 0 0 4 0 4 0 2 2 1 0 4 4 4 2 0 4 1 0 4 4 4 0 4 0 4 4 4
 1 2 1 2 3 4 1 4 4 4 4 4 0 0 4 4 4 0 0 2 4 4 4 4 4 4 1 1 1 4 2 4 1 4 1 4 4
 3 4 3 0 2 4 0 1 1 2 4 1 4 4 4 2 1 4 4 4 4 3 1 2 4 4 2 4 1 2 1 1 2 0 4 4 3
 4 0 0 4 4 3 1 2 2 2 4 4 0 4 4 0 1 0 4 1 0 4 4 1 0 4 4 1 1 1 0 2 4 2 1 1 0
 4 1 4 0 0 4 1 2 1 4 4 4 4 4 0 4 4 0 4 0 4 4 1 0 4 4 0 4 2 0 4 1 4 4 1 4 2
 1 0 1 0 2 2 4 3 4 2 0 1 0 2 1 4 2 4 2 0 4 4 4 0 0 2 0 0 1 0 0 4 4 4 0 2 4
 4 2 4 4 1 4 4 4 4 4 4 2 0 1 2 4 4 0 4 2 2 2 4 4 2 1 2 4 0 4 4 2 1 4 4 1 0
 1 4 0 4 4 4 0 0 2 0 4 2 4 0 0 2 4 1 2 0 2 2 4 4 4 0 4 4 4 4 2 4 4 1 4 4 2
 1 4 2 2 0 4 0 0 4 4 4 2 1 4 4 1 0 4 4 4 0 4 0 1 3 1 4 4 0 4 4 2 1 4 2 1 2
 4 4 4 0 4 0 3 4 1 4 0 0 4 2 2 4 4 4 0 1 4 1 2 4 4 1 4 2 4 4 4 1 4 2 2 4 0
 4 4 4 4 4 2 2 3 4 4 1 4 4 4 4 2 2 4 4 0 1 4 2 1 4 0 4 2 4 4 4 4 2 2 1 4 4
 0 4 4 3 0 0 4 2 0 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4
 4 2 4 4]


In [ ]:
accuracy_score(y_true, y_pred)

0.17938144329896907

In [ ]:
print(classification_report(y_true, y_pred,target_names=label_set,zero_division=1))

              precision    recall  f1-score   support

     produto       0.69      0.17      0.27       360
     entrega       0.35      0.25      0.29       102
      imagem       0.00      0.00      0.00         3
  desempenho       0.00      0.00      0.00        19
         som       0.00      1.00      0.01         1

    accuracy                           0.18       485
   macro avg       0.21      0.28      0.11       485
weighted avg       0.59      0.18      0.26       485



## Avaliações com o my_word_vectors gerado

In [ ]:
GAMMA = .03
N_ASPECT_WORDS = 200

scores = defaultdict(dict)
r = Reach.load("/content/my_word_vectors.vec",unk_word="<UNK>")

aspects = [[x] for x in json.load(open("/content/aspect_words.json"))]
aspects = aspects[:N_ASPECT_WORDS]
instances = df_tst['text']
# instances = ["a comida estava deliciosa e muito bem temperada".lower().split(), 
#               "o atendimento estava incrível, recomendo demais".lower().split(), 
#               "um ambiente de otima qualidade".lower().split(),
#               "o lugar é muito bem localizado".lower().split(),
#               "achei o preco um pouco caro mas vale a pena".lower().split()]
#instances = df['text'][5:10]
#print(instances)
label_set = ['produto', 'entrega', 'imagem', 'desempenho', 'som']
#[i.lower().split() for i in instances]
s = get_scores([i.lower().split() for i in instances],
               aspects,
               r,
               label_set,
               gamma=GAMMA,
               remove_oov=False,
               attention_func=rbf_attention)

y_pred = s.argmax(1)
#print(s)
print(y_pred)

['produto', 'entrega', 'imagem', 'desempenho', 'som']
[2 2 2 2 2 4 0 0 2 2 0 2 0 2 2 4 0 2 2 2 2 0 4 2 0 2 2 0 2 2 4 2 2 2 2 4 2
 2 1 0 2 2 0 0 2 0 0 2 0 2 2 4 2 2 0 2 0 2 2 2 2 2 4 0 2 2 2 2 0 2 2 0 2 2
 2 2 1 2 0 2 0 2 2 2 2 2 2 0 0 4 2 0 2 2 2 2 2 2 2 4 0 0 0 4 2 2 2 2 2 4 2
 4 2 0 2 2 2 2 0 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 0 4 4 2
 2 2 0 2 2 4 1 2 0 2 2 2 2 4 2 2 0 0 2 1 2 2 2 2 2 2 2 4 0 2 2 4 2 2 0 2 0
 2 4 2 2 2 2 2 2 2 2 4 2 2 2 0 2 0 0 0 2 2 4 2 2 0 2 0 2 2 0 0 2 2 2 0 2 2
 0 0 2 0 2 2 2 2 4 2 0 2 2 2 0 0 4 0 0 0 2 4 2 0 2 2 0 2 2 4 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 4 4 0 2 2 0 2 2 0 0 4 2 2 2 4 4 2 2 2 2 2 2 2 2 1 2 4 2 0
 2 2 0 2 2 2 2 0 4 2 2 2 2 0 0 2 2 2 2 2 2 2 4 4 4 2 2 2 2 2 0 4 2 2 2 2 2
 2 2 2 2 2 2 0 0 0 2 4 2 0 2 0 2 0 2 0 2 0 4 0 2 2 0 2 2 2 4 0 0 2 4 0 0 2
 0 0 2 2 2 0 2 4 2 2 0 0 2 2 2 2 2 2 2 0 2 2 2 4 2 2 4 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 4 2 2 4 2 2 2 2 2 2 2 2 2 4 2 4 2 2 2 2 2 4 2 4
 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 4 4

In [ ]:
accuracy_score(y_true, y_pred)

0.09896907216494845

In [ ]:
print(classification_report(y_true, y_pred,target_names=label_set,zero_division=1))

              precision    recall  f1-score   support

     produto       0.52      0.13      0.21       360
     entrega       0.00      0.00      0.00       102
      imagem       0.01      0.67      0.01         3
  desempenho       1.00      0.00      0.00        19
         som       0.00      0.00      0.00         1

    accuracy                           0.10       485
   macro avg       0.31      0.16      0.04       485
weighted avg       0.43      0.10      0.15       485



# Funções auxiliares

In [ ]:
with open('/content/restaurant_textos.txt ', 'w') as textfile:
  textfile.write("\n".join(str(item) for item in sentences))
textfile.close()

In [ ]:
with open('/content/corpus.txt ', 'w') as textfile:
  for item in corpus:
    textfile.write(str(item))
textfile.close()

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

def colorize(words, color_array):
    cmap=matplotlib.cm.Blues
    template = '<span class="barcode"; style="color: black; background-color: {}">{}</span>'
    colored_string = ''
    for word, color in zip(words, color_array):
        color = matplotlib.colors.rgb2hex(cmap(color))
        print(color)
        colored_string += template.format(color, '&nbsp' + word + '&nbsp')
    return colored_string

words = instances[0]#['um', 'ambiente', 'de', 'ótima', 'qualidade']
color_array = atts[0][0]

print(color_array)
s = colorize(words, color_array)

# or simply save in an html file and open in browser
with open('/content/colorize.html', 'w') as f:
    f.write(s)

[0.1934354  0.19726068 0.19610389 0.20825753 0.20494252]
#d1e2f3
#d0e2f2
#d0e2f2
#cee0f2
#cfe1f2
